<a href="https://colab.research.google.com/github/zahraDehghanian97/DenseNet_Vs._ResNet/blob/master/Densenet_Vs__ResNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Mount google drive**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **prerequisit**

In [16]:
%tensorflow_version 2.x
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import random
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn import preprocessing
from tensorflow.keras.preprocessing import image
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Input, GlobalAveragePooling2D, Dropout ,  BatchNormalization
import glob
from sklearn.model_selection import train_test_split
from keras.models import Sequential
import os
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


# **call back**

In [ ]:
class EarlyStoppingCallback(keras.callbacks.Callback):
  def __init__(self,patience=0):
    super(EarlyStoppingCallback,self).__init__()
    self.patience=patience
    self.best=np.Inf
    self.wait=0
    self.stopped_epoch=0
  def on_epoch_end(self,epoch,logs=None):
    current_loss=logs.get("val_loss")
    if np.less(current_loss,self.best):
      self.best=current_loss
      self.wait=0
      self.best_weights=self.model.get_weights()
    else:
      self.wait+=1
      print("\nwait mode, step: %d"% self.wait)
      if self.wait>=self.patience:
        self.stopped_epoch=epoch
        self.model.stop_training=True
        self.model.set_weights(self.best_weights)
        print("epoch: %d : early stopping."% self.stopped_epoch)
        self.wait = 0
      

es_callback=EarlyStoppingCallback(patience=5)

# **Make dataset ready**

In [ ]:
def load_photos(type,dir_name):
    photo_list =[]
    y = []
    for file_name in (glob.glob(dir_name+'/*')):
        img = image.load_img(file_name, target_size=input_size)
        img = np.array(img)
        photo_list.append(img)
        y.append(type)
    return photo_list , y 
  
    
input_size = (250,250)
input_shape = (250,250,3)
dir_name_indoor = "/content/drive/MyDrive/Colab Notebooks/indoor"
dir_name_outdoor = "/content/drive/MyDrive/Colab Notebooks/outdoor"
X1 , y1= load_photos(0,dir_name_indoor) 
X2 , y2= load_photos(1,dir_name_outdoor) 
X1.extend(X2)
y1.extend(y2)
X_train, X_test, y_train, y_test = train_test_split(X1, y1, test_size=0.1, random_state=42)
X_train = np.array(X_train)
X_test = np.array(X_test)
y_test = np.array(y_test)
y_train = np.array(y_train)

# **ResNet**

In [ ]:
keras.backend.clear_session() 

base_model = keras.applications.ResNet152(include_top=False, weights="imagenet",  input_shape=input_shape)
base_model.trainable  = False
model_res = Sequential()
model_res.add(base_model)
model_res.add(Flatten())
model_res.add(Dense(1500,activation='relu'))
model_res.add(Dropout(0.2))
model_res.add(Dense(500,activation='relu'))
model_res.add(Dropout(0.1))
model_res.add(Dense(150,activation='relu'))
model_res.add(Dense(50,activation='relu'))
model_res.add(Dense(10,activation='relu'))
model_res.add(Dense(2))
tf.keras.utils.plot_model(model_res,show_shapes=True,expand_nested=True)

 

In [ ]:
model_res.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])
tb_callback = keras.callbacks.TensorBoard(log_dir="./logs",histogram_freq=1)
model_res.fit(X_train,y_train,epochs=1000,validation_split=0.2,callbacks=[es_callback,tb_callback])

Epoch 1/1000


In [13]:
prediction_without_fine_tune_res =np.argmax(model_res.predict(X_test),axis=1)

fine tune

In [15]:
layers = base_model.layers
for layer in layers[-50:-1]:
  layer.trainable = True
model_res_fine=keras.models.Sequential()
model_res_fine.add(base_model)
for i in range(1,len(layers)):
  model_res_fine.add(model_res.layers[i])
model_res_fine.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001), loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])
model_res_fine.fit(X_train,y_train,epochs=1000,validation_split=0.2,callbacks=[es_callback,tb_callback])

Epoch 1/1000
18/18 [==============================] - 419s 23s/step - loss: 0.1655 - accuracy: 0.9802 - val_loss: 2.6000 - val_accuracy: 0.9236

wait mode, step: 6


ValueError: ignored

In [ ]:
prediction_fine_tune_res=np.argmax(model_res_fine.predict(X_test),axis=1)
print("ResNet Result ==============================>>")
print('accuracy without fine tuning = '+ str(accuracy_score(y_test,prediction_without_fine_tune_res)))
print('confusion matrix without fine tuning'+ str( confusion_matrix(y_test,prediction_without_fine_tune_res)))
print('accuracy with fine tuning = '+ str(accuracy_score(y_test,prediction_fine_tune_res)))
print('confusion matrix with fine tuning'+ str( confusion_matrix(y_test,prediction_fine_tune_res)))


# **DenseNet**

In [ ]:
keras.backend.clear_session() 

base_model = keras.applications.DenseNet201(include_top=False, weights="imagenet",  input_shape=input_shape)
base_model.trainable  = False
model_dense = Sequential()
model_dense.add(base_model)
model_dense.add(Flatten())
model_dense.add(Dense(1500,activation='relu'))
model_dense.add(Dropout(0.2))
model_dense.add(Dense(500,activation='relu'))
model_dense.add(Dropout(0.1))
model_dense.add(Dense(150,activation='relu'))
model_dense.add(Dense(50,activation='relu'))
model_dense.add(Dense(10,activation='relu'))
model_dense.add(Dense(2))
tf.keras.utils.plot_model(model_dense,show_shapes=True,expand_nested=True)

 

In [ ]:
model_dense.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])
tb_callback = keras.callbacks.TensorBoard(log_dir="./logs",histogram_freq=1)
model_dense.fit(X_train,y_train,epochs=1000,validation_split=0.2,callbacks=[es_callback,tb_callback])

In [ ]:
prediction_without_fine_tune_dense =np.argmax(model_res.predict(X_test),axis=1)

fine tune

In [ ]:
layers = base_model.layers
for layer in layers[-50:-1]:
  layer.trainable = True
model_dense_fine=keras.models.Sequential()
model_dense_fine.add(base_model)
for i in range(1,len(layers)):
  model_dense_fine.add(model_dense.layers[i])
model_dense_fine.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001), loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])
model_dense_fine.fit(X_train,y_train,epochs=1000,validation_split=0.2,callbacks=[es_callback,tb_callback])

In [ ]:
prediction_fine_tune_dense=np.argmax(model_dense_fine.predict(X_test),axis=1)
print("DenseNet Result =====================>>>>")
print('accuracy without fine tuning = '+ str(accuracy_score(y_test,prediction_without_fine_tune_dense)))
print('confusion matrix without fine tuning'+ str( confusion_matrix(y_test,prediction_without_fine_tune_dense)))
print('accuracy with fine tuning = '+ str(accuracy_score(y_test,prediction_fine_tune_dense)))
print('confusion matrix with fine tuning'+ str( confusion_matrix(y_test,prediction_fine_tune_dense)))


# **Comparison**

In [ ]:
if prediction_fine_tune_dense> prediction_fine_tune_res :
  print("DenseNet Result is better")
else : 
  print("ResNet Result is better")
  
print("============================")
print('accuracy DenseNet = '+ str(accuracy_score(y_test,prediction_fine_tune_dense)))
print('accuracy ResNet = '+ str(accuracy_score(y_test,prediction_fine_tune_res)))